In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [2]:
US = pd.read_csv('Data/Capstone/US-ok.csv')
CA = pd.read_csv('Data/Capstone/CA-ok.csv')
MX = pd.read_csv('Data/Capstone/MX-ok.csv')

In [3]:
NA = pd.concat([CA, US, MX], axis = 0)
NA.corr()

,category_id,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,assignable
category_id,1.000000,-0.148186,-0.157194,-0.033730,-0.074211,0.028423,0.012376,-0.027876,-0.128189
views,-0.148186,1.000000,0.840269,0.491227,0.638893,0.006470,0.000894,-0.000951,0.004455
likes,-0.157194,0.840269,1.000000,0.448215,0.815388,-0.021172,-0.024184,-0.001086,0.008139
dislikes,-0.033730,0.491227,0.448215,1.000000,0.676530,-0.003396,-0.009207,-0.000697,0.002482
comment_count,-0.074211,0.638893,0.815388,0.676530,1.000000,-0.023267,-0.016678,-0.001111,0.006356
comments_disabled,0.028423,0.006470,-0.021172,-0.003396,-0.023267,1.000000,0.232254,-0.002868,0.004628
ratings_disabled,0.012376,0.000894,-0.024184,-0.009207,-0.016678,0.232254,1.000000,-0.002202,0.003555
video_error_or_removed,-0.027876,-0.000951,-0.001086,-0.000697,-0.001111,-0.002868,-0.002202,1.000000,0.000968
assignable,-0.128189,0.004455,0.008139,0.002482,0.006356,0.004628,0.003555,0.000968,1.000000


In [4]:
NA.isna().sum()
#NorAm = NA[NA['views'] >= 5000000].copy() Veeeery Important
NorAm = NA.copy()

In [5]:
import time
NorAm['trending_date'] = NorAm.trending_date.apply(lambda date : '20' + str(date))
NorAm['trending_date'] = NorAm.trending_date.apply(lambda date : str(date).replace('.', '/'))
# Converting Trending date and publish time to datetime format
NorAm.trending_date = pd.to_datetime(NorAm['trending_date'], format = '%Y/%d/%m')
NorAm.publish_time = pd.to_datetime(NorAm['publish_time'])

NorAm['NoTags'] = [len(tag.split('|')) for tag in NorAm['tags']]
NorAm['NoWord'] = [len(title.split(' ')) for title in NorAm['title']]
NorAm['NoChar'] = [len(title.strip(' ')) for title in NorAm['title']]
NorAm['Months'] = NorAm['trending_date'].dt.month.astype(object)

NorAm.drop(['video_id', 'category_id', 'thumbnail_link', 'description', 'title', 'channel_title', 'tags'], axis = 1, inplace = True )
NorAm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121858 entries, 0 to 40149
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   trending_date           121858 non-null  datetime64[ns]     
 1   publish_time            121858 non-null  datetime64[ns, UTC]
 2   views                   121858 non-null  int64              
 3   likes                   121858 non-null  int64              
 4   dislikes                121858 non-null  int64              
 5   comment_count           121858 non-null  int64              
 6   comments_disabled       121858 non-null  bool               
 7   ratings_disabled        121858 non-null  bool               
 8   video_error_or_removed  121858 non-null  bool               
 9   category                121858 non-null  object             
 10  assignable              121858 non-null  bool               
 11  NoTags                  121

In [6]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['category', 'Months'])
NAbe = encoder.fit_transform(NorAm)

In [7]:
NAbe = pd.get_dummies(NAbe, drop_first = True)
NAbe.shape

(121858, 23)

In [8]:
Xn = NAbe.drop(['views','trending_date', 'publish_time'], axis= 1)
Yn = NAbe['views']

In [9]:
from sklearn.feature_selection import SelectKBest, f_regression
feat = SelectKBest(score_func=f_regression, k='all')
Selectd_X = feat.fit(Xn, Yn)
#print(Selectd_X.scores_)
score = pd.DataFrame(Selectd_X.scores_)
feat = pd.DataFrame(Xn.columns)
FeatScores = pd.concat([feat, score], axis =1)
FeatScores.columns=['Features','Score']
print(FeatScores.nlargest(NAbe.shape[1],'Score'))

Features          Score
0                    likes  292694.276138
2            comment_count   84045.730989
1                 dislikes   38756.316012
9               category_3     921.130304
11              category_5     715.918984
10              category_4     515.029282
15                  NoChar     435.872499
16                Months_0     406.173222
14                  NoWord     349.156461
8               category_2     130.497618
17                Months_1     122.694769
13                  NoTags      42.421514
18                Months_2      33.465405
7               category_1       7.327577
3        comments_disabled       5.100821
12              assignable       2.418672
19                Months_3       2.074668
5   video_error_or_removed       0.110149
4         ratings_disabled       0.097481


In [10]:
NAbe1 = NAbe[FeatScores['Features'][FeatScores['Score'] > 2]]
NAbe1['views'] = NAbe['views'] 
NAbe1 = NAbe1[NAbe1['views'] >= 5000000]
#FeatScores['Features'][FeatScores['Score'] > 2]

In [11]:
Xn = NAbe1.drop('views', axis =1)
Yn = np.log(NAbe1['views'])

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(Xn)
y = sc.fit_transform(Yn.values.reshape(-1, 1))

from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(Xn, Yn, test_size = 0.3, random_state=42)

In [12]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

model1 = lm.fit(train_X, train_y)
y_pred = lm.predict(test_X)

from sklearn.metrics import mean_squared_error as mse
k = mse(y_pred, test_y)
rsme = mse(y_pred, test_y)**(1/2)
round(rsme, 2), round(k, 2)

(0.47, 0.22)

In [13]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=200, max_depth= 100)
model2 = rfr.fit(train_X, train_y)
y_pred = rfr.predict(test_X)

k = mse(y_pred, test_y)
rsme = mse(y_pred, test_y)**(1/2)
round(rsme, 2), round(k, 2)

(0.21, 0.04)

In [14]:
from sklearn.svm import SVR

svr = SVR(kernel='rbf', degree = 3)
model3 = svr.fit(train_X, train_y)
y_pred = svr.predict(test_X)

k = mse(y_pred, test_y)
rsme = mse(y_pred, test_y)**(1/2)
round(rsme, 2), round(k, 2)

(0.45, 0.2)

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
params = {'n_neighbors':list(range(1,21))}
knn = KNeighborsRegressor()
model = GridSearchCV(knn, params, cv=5)
model.fit(train_X, train_y)
print(model.best_params_)

y_pred2 = model.predict(test_X)
k = mse(test_y, y_pred2)
rsme = mse(y_pred, test_y)**(1/2)
round(rsme, 2), round(k, 2)

{'n_neighbors': 3}


(0.45, 0.14)